# Reservation Summary

###### Combining listing events and reservation, I created a table called ReservationFact. This table contains the total number of times the listing appeared in a search.  If it appeared in a search, I then looked for a record in the reservation table if the property was reserved. 

Removed is_bot and is_host records

In [0]:
%sql
select * from neighbor.silver.reserverationfact where ListingDMA='9-787'
Limit 10

In [0]:
%sql
SELECT
DMAName,
Sum(TotalSearchCount) as TotalSearches,
Sum(Reserved) as TotalReservations,
Sum(NotReserved) as TotalNotReserved,
ROUND((Sum(Reserved) / Sum(TotalSearchCount)*100),2)  as ConversionRate,
ROUND((Sum(NotReserved) / Sum(TotalSearchCount)*100),2)  as BrowseRate
from neighbor.silver.reserverationfact
group by DMAName
order by conversionrate DESC

#### So lets peel back the layers and look at the conversion rates at the attribution rate


In [0]:
%sql
SELECT
first_attribution_source  as Source,
Sum(TotalSearchCount) as TotalSearches,
Sum(Reserved) as TotalReservations,
Sum(NotReserved) as TotalNotReserved,
ROUND((Sum(Reserved) / Sum(TotalSearchCount)*100),2)  as ConversionRate,
ROUND((Sum(NotReserved) / Sum(TotalSearchCount)*100),2)  as BrowseRate
from neighbor.silver.reserverationfact
group by first_attribution_source

#### Lets look at the channel 

In [0]:
%sql
SELECT
first_attribution_channel  as Channel,
Sum(TotalSearchCount) as TotalSearches,
Sum(Reserved) as TotalReservations,
Sum(NotReserved) as TotalNotReserved,
ROUND((Sum(Reserved) / Sum(TotalSearchCount)*100),2)  as ConversionRate,
ROUND((Sum(NotReserved) / Sum(TotalSearchCount)*100),2)  as BrowseRate
from neighbor.silver.reserverationfact
group by first_attribution_channel
order by conversionrate DESC

#### Lets combine Source and Channel and look for some nuances

In [0]:
%sql
SELECT
first_attribution_source as Source,
first_attribution_channel  as Channel,
Sum(TotalSearchCount) as TotalSearches,
Sum(Reserved) as TotalReservations,
Sum(NotReserved) as TotalNotReserved,
ROUND((Sum(Reserved) / Sum(TotalSearchCount)*100),2)  as ConversionRate,
ROUND((Sum(NotReserved) / Sum(TotalSearchCount)*100),2)  as BrowseRate
from neighbor.silver.reserverationfact
group by 
first_attribution_source,
first_attribution_channel
order by conversionrate DESC

Some interesting differences in the conversion rate in the channel data. At an aggregate, Reddit is the lowest and Google Maps is the highest  Lets do a Pivot and see how the conversion rate by DMA by channel compares

In [0]:
%sql
SELECT
DMAName,
first_attribution_channel  as Channel,
ROUND((Sum(Reserved) / Sum(TotalSearchCount)*100),2)  as ConversionRate
from neighbor.silver.reserverationfact 
group by 
DMAName,
first_attribution_channel


Databricks visualization. Run in Databricks to view.

####Lets look at the number of reservations at an aggregate

In [0]:
%sql
WITH property_reservations AS (
    SELECT
        DMAName,
        ListingDMA,
        SUM(Reserved) AS total_reservations
    FROM neighbor.silver.reserverationfact
    GROUP BY DMAName,ListingDMA
)
SELECT
    DMAName,
    COUNT(ListingDMA) as TotalProperties,
    COUNT(CASE WHEN total_reservations = 0 THEN 1 END) AS ZeroReservations,
    COUNT(CASE WHEN total_reservations = 1 THEN 1 END) AS OneReservation,
    COUNT(CASE WHEN total_reservations = 2 THEN 1 END) AS TwoReservations,
    COUNT(CASE WHEN total_reservations = 3 THEN 1 END) AS ThreeReservations,
    COUNT(CASE WHEN total_reservations = 4 THEN 1 END) AS FourReservations,
    COUNT(CASE WHEN total_reservations >= 5 THEN 1 END) AS FivePlusReservations
FROM property_reservations
GROUP BY DMAName;

In [0]:
%sql
With resData as (
select 
ListingDMA,
--first_attribution_source,
--first_attribution_channel,
count (distinct ListingDMA) as Listings,
sum(AppearedInSearches) as totalSearches,
Avg(AvgSearchPosition) as avgSearchPosition,
sum(Reserved) as reserved,
sum(NotReserved) as notReserved
from  neighbor.silver.reserverationfact --where ListingDMA='3-121'
group by 
ListingDMA--,
--first_attribution_source,
--first_attribution_channel
)

select 
'Zero Reservations' as ReservationCount,
count(distinct ListingDMA) as Listings,
Round(avg(totalSearches),0)  as AvgSearchReturns,
Round(Avg(avgSearchPosition),2) as avgSearchPosition
from resData where reserved=0
union all
select 
'One Reservation' as ReservationCount,
sum(Listings) as Listings,
Round(avg(totalSearches),0)  as AvgSearchReturns,
Round(Avg(avgSearchPosition),2) as avgSearchPosition
from resData where reserved=1
union all
select 
'Two Reservations' as ReservationCount,
sum(Listings) as Listings,
Round(avg(totalSearches),0)  as AvgSearchReturns,
Round(Avg(avgSearchPosition),2) as avgSearchPosition
from resData where reserved=2
union all
select 
'Three Reservations' as ReservationCount,
sum(Listings) as Listings,
Round(avg(totalSearches),0)  as AvgSearchReturns,
Round(Avg(avgSearchPosition),2) as avgSearchPosition
from resData where reserved=3
union all
select 
'Four Reservations' as ReservationCount,
sum(Listings) as Listings,
Round(avg(totalSearches),0)  as AvgSearchReturns,
Round(Avg(avgSearchPosition),2) as avgSearchPosition
from resData where reserved=4
union all
select 
'Five+ Reservations' as ReservationCount,
sum(Listings) as Listings,
Round(avg(totalSearches),0)  as AvgSearchReturns,
Round(Avg(avgSearchPosition),2) as avgSearchPosition
from resData where reserved>4




##SEARCH DMA SUMMARY

In [0]:
## Load Python libraries 
%pip install folium

In [0]:
%sql

select 
c.DMAName,
count(distinct a.MergedAmplitudeID) as DistinctUsers,
count(distinct a.SearchID) as TotalSearches,
count(distinct a.priorsearchid) as PriorSearches,
count(distinct a.listingID) as ListingCount,
round(avg(a.SearchCounts),0) as AvgPropertiesReturned,
round(avg(AvgSearchPositionClicked),0) as AvgClickPosition,
sum(PropertiesClicked) as PropertiesClicked,
round((sum(a.PropertiesClicked) /  count(distinct a.SearchID)),2) as AvgPropertiesClicked

 from neighbor.silver.searchevents a join
neighbor.silver.searchterm b on a.SearchTermID=b.SearchTermID
join neighbor.silver.dma c on a.DMAID=c.DMAID
join neighbor.silver.channel d on a.channelID=d.channelID
join neighbor.silver.time e on a.TimeKey=e.TimeKey
join neighbor.silver.source f on a.SourceID=f.SourceID
join neighbor.silver.calendar g on a.CalendarKey=g.CalendarKey

group by c.DMAName

In [0]:
from folium.plugins import MarkerCluster

# Center the map at the mean location
center_lat = pdf['latitude'].mean()
center_lon = pdf['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Add clustered points
marker_cluster = MarkerCluster().add_to(m)
for _, row in pdf.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(marker_cluster)

display(m)

In [0]:
%sql
SELECT
b.state,
a.search_type,
search_dma,
count(distinct a.search_id) as TotalSearches
FROM 
neighbor.bronze.all_search_events a JOIN
     neighbor.bronze.us_state_bounding_boxes b
     on a.latitude between b.min_lat and b.max_lat
     and a.longitude between b.min_lon and b.max_lon
group by 
b.state,
a.search_type,
search_dma
order by state

In [0]:
%sql
SELECT 
  b.state AS stage1,
  search_term AS stage3,
  COUNT(DISTINCT search_id) AS value
FROM 
  neighbor.bronze.all_search_events a
JOIN
  neighbor.bronze.us_state_bounding_boxes b
  ON a.latitude BETWEEN b.min_lat AND b.max_lat
  AND a.longitude BETWEEN b.min_lon AND b.max_lon
GROUP BY
  state,
  search_term
ORDER BY
  value DESC


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
  b.state AS `State Origin`,
  search_term AS `Search Term`,
  search_dma AS `Search DMA`,
  COUNT(DISTINCT search_id) AS value
FROM 
  neighbor.bronze.all_search_events a
JOIN
  neighbor.bronze.us_state_bounding_boxes b
  ON a.latitude BETWEEN b.min_lat AND b.max_lat
  AND a.longitude BETWEEN b.min_lon AND b.max_lon
GROUP BY
  state,
  search_term,
  search_dma
ORDER BY
  value DESC


Databricks visualization. Run in Databricks to view.